# SWIS-100 Batch run

This is an illustrative notebook for running a SWIS-100 "experiment" - a batch of single runs, driven from an array of run configurations provided via a single external (hard coded) .ods spreadsheet file. It provides a notebook-based alternative to the command line script version. In principle, it allows batch runs via the [mybinder.org](http://www.mybinder.org) cloud service; but note that in that specific use case, it is essential to download the output file(s) before the mybinder.org session terminates.


In [ ]:
import os
import numpy as np
import pandas as pd
import swis100 as swis


In [ ]:
batch_configs = pd.read_excel('batch-run-example/batch_config.ods',
                              header=0,
                              index_col=0,
                              sheet_name='swis-config',
                              converters=swis.config_converters
                             )

display(batch_configs)


In [ ]:
batch_configs_dict = batch_configs.to_dict(orient='index')

display(batch_configs_dict)

In [ ]:
batch_dir='batch-run-example'
os.makedirs(batch_dir,exist_ok=True) # Precautionary


## Solve the system (batch-run, no parallelization support)

In [ ]:
batch_stats = pd.DataFrame(dtype=object)
for run_id in batch_configs_dict.keys() :
    display(run_id)
    run_config = batch_configs_dict[run_id]
    network = swis.solve_network(run_config)
    
    #network.export_to_netcdf(batch_dir+'/'+run_id+'-network.nc') 
        # Uncomment if full network object data should be saved; but note that
        # SWIS-100-IE does not provide any pre-built tools to further view or process such files.
    
    batch_stats[run_id] = swis.gather_run_stats(run_config, network)

In [ ]:
with pd.option_context('display.max_rows', None): # Display ALL rows (no ellipsis)
    display(batch_stats)

## Dump (volatile) run data to (persistent) file storage


In [ ]:

#run_config_series=pd.Series(run_config, dtype=object, name=run_id)
#batch_configs.to_excel(batch_dir+'/batch_config.ods')
batch_stats.to_excel(batch_dir+'/batch_stats.ods')
